# Plan and Execute

> - 참고 자료: https://wikidocs.net/270688
> - 관련 논문: https://arxiv.org/abs/2305.04091

Plan-and-Execute 에이전트는 먼저 달성할 목표를 구체적인 단계로 나누어 계획을 수립한 뒤, 그 계획을 순차적으로 실행하며 실행 과정에서 얻은 피드백을 반영해 동적으로 조정하는 구조를 갖추고 있습니다.

[ReAct](https://arxiv.org/abs/2210.03629) 에이전트는 사고와 행동을 매 단계 즉시 교차 실행하며 즉각적 피드백에 대응하는 반면, Plan-and-Execute 에이전트는 전체 목표를 세부 단계별로 계획 수립 후 순차 실행하며 피드백으로 계획을 동적으로 조정합니다.

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("../.env", override=True)


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"
_set_env("OPENAI_API_KEY")

## 에이전트 정의

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_tavily import TavilySearch

web_search_tool = TavilySearch(max_results=3)

agent_executor = create_react_agent(
    model="openai:gpt-4.1-mini",
    tools=[web_search_tool],
    prompt="당신은 도움이 되는 유능한 조수입니다.",
)

In [8]:
agent_executor.invoke({"messages": [("user", "랭그래프에 대해서 웹검색 해주세요.")]})

{'messages': [HumanMessage(content='랭그래프에 대해서 웹검색 해주세요.', additional_kwargs={}, response_metadata={}, id='d80abbf7-b5fc-4246-87c0-b8ecf746cbac'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_voTk7cm4nXxdYQs6lYK9mH9c', 'function': {'arguments': '{"query":"랭그래프"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 1279, 'total_tokens': 1298, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1152}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_95d112f245', 'id': 'chatcmpl-CP6FsGgGXT2ABnaemzFMyzHUgMlCl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ff4808c9-d741-4fb5-ac41-a1e8f2715d18-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '랭그래프'}, 'id': 'cal